In [1]:
%run liblecture.py

In [2]:
movies = pd.read_csv('./data/movielens/movielens/movies_w_imgurl.csv')
movies.head()

,movieId,imdbId,title,genres,imgurl
0,1,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://images-na.ssl-images-amazon.com/images...
1,2,113497,Jumanji (1995),Adventure|Children|Fantasy,https://images-na.ssl-images-amazon.com/images...
2,3,113228,Grumpier Old Men (1995),Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,4,114885,Waiting to Exhale (1995),Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,5,113041,Father of the Bride Part II (1995),Comedy,https://images-na.ssl-images-amazon.com/images...


In [3]:
ratings = pd.read_csv('./data/movielens/movielens/ratings-9_1.csv')

train = ratings[ratings['type']=='train'][['userId', 'movieId', 'rating']]
test = ratings[ratings['type']=='test'][['userId', 'movieId', 'rating']]

In [5]:
# create index to id maps
movieIds = train.movieId.unique()

movieIdToIndex = {}
indexToMovieId = {}

rowIdx = 0

for movieId in movieIds:
    movieIdToIndex[movieId] = rowIdx
    indexToMovieId[rowIdx] = movieId
    rowIdx += 1

In [7]:
userIds = train.userId.unique()

userIdToIndex = {}
indexToUserId = {}

colIdx = 0

for userId in userIds:
    userIdToIndex[userId] = colIdx
    indexToUserId[colIdx] = userId
    colIdx += 1

In [8]:
# create item-user sparse matrix
import scipy.sparse as sp

rows = []
cols = []
vals = []

for row in train.itertuples():
    rows.append(movieIdToIndex[row.movieId])
    cols.append(userIdToIndex[row.userId])
    vals.append(row.rating)

coomat = sp.coo_matrix((vals, (rows, cols)), shape=(rowIdx, colIdx))

matrix = coomat.todense()
matrix

matrix([[2.5, 0. , 0. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [2. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [9]:
# compute l2-norm
from numpy import linalg as LA

norms = LA.norm(matrix, ord=2, axis=1)
norms

array([20.71231518, 20.34698995, 22.94013949, ...,  3.        ,
        1.        ,  1.        ])

In [10]:
normmat = np.divide(matrix.T, norms).T
normmat

matrix([[0.12070114, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.14744196, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.08718343, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [12]:
sims = pd.DataFrame(data=np.matmul(normmat, normmat.T), index=movieIds, columns=movieIds)
sims

,31,1061,1129,1172,1287,1293,1339,1343,1371,1405,...,134528,134783,137595,138204,60832,64997,72380,129,4736,6425
31,1.000000,0.201693,0.125752,0.070271,0.086726,0.145710,0.070816,0.183679,0.124345,0.140467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1061,0.201693,1.000000,0.249591,0.181847,0.140887,0.131740,0.253097,0.200022,0.158972,0.228312,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.147442,0.147442,0.0
1129,0.125752,0.249591,1.000000,0.166642,0.253575,0.162663,0.264157,0.295043,0.335743,0.310540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1172,0.070271,0.181847,0.166642,1.000000,0.267011,0.326330,0.117903,0.124118,0.109589,0.157045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1287,0.086726,0.140887,0.253575,0.267011,1.000000,0.315892,0.173386,0.073288,0.184039,0.183198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.149889,0.000000,0.000000,0.143674,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.000000,0.000000,0.0
72380,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.149889,0.000000,0.000000,0.143674,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.000000,0.000000,0.0
129,0.000000,0.147442,0.000000,0.000000,0.000000,0.000000,0.128476,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0
4736,0.000000,0.147442,0.000000,0.000000,0.000000,0.000000,0.128476,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0


## Example

In [18]:
movieIdx = 6

rels = sims.iloc[movieIdx,:].sort_values(ascending=False).head(10)[1:]

displaymovies(movies, [indexToMovieId[movieIdx]])
# 해당 영화와 유사한 영화
displaymovies(movies, rels.index, rels.values)

In [19]:
userId = 33

userRatings = train[train['userId']==userId][['movieId', 'rating']]

userRatings

,movieId,rating
6176,19,3.0
6177,88,3.0
6178,157,1.0
6179,231,3.0
6180,344,4.0
...,...,...
6309,5282,4.0
6310,5339,4.0
6311,5483,4.0
6312,5669,4.0


In [20]:
# predict item ratings for the test users
recSimSums = sims.loc[userRatings['movieId'].values, :].sum().values

recWeightedRatingSums = np.matmul(sims.loc[userRatings['movieId'].values, :].T.values, userRatings['rating'].values)

recItemRatings = pd.DataFrame(np.divide(recWeightedRatingSums, recSimSums), index=sims.index)

recItemRatings.columns = ['pred']

recItemRatings

C:\Users\wkddn\AppData\Local\Temp\ipykernel_64372\2997029718.py:6: RuntimeWarning: invalid value encountered in true_divide
  recItemRatings = pd.DataFrame(np.divide(recWeightedRatingSums, recSimSums), index=sims.index)


,pred
31,3.428552
1061,3.381315
1129,3.362303
1172,3.490657
1287,3.421083
...,...
64997,3.228323
72380,3.228323
129,2.997086
4736,2.997086


In [21]:
top30Movies = recItemRatings.sort_values(by='pred', ascending=False).head(30) 

displaymovies(movies, top30Movies.index, top30Movies['pred'].values)

In [22]:
userTestRatings = pd.DataFrame(data=test[test['userId']==userId])

temp = userTestRatings.join(recItemRatings.loc[userTestRatings['movieId']], on='movieId')

mae = getMAE(temp['rating'], temp['pred'])
rmse = getRMSE(temp['rating'], temp['pred'])

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

MAE: 0.7424
RMSE: 0.8455


- mae와 rmse가 전에 비해 현저히 줄어든 것을 확인할 수 있음

In [23]:
logs = userRatings.sort_values(by='rating', ascending=False).head(20)
recs = recItemRatings.sort_values(by='pred', ascending=False).head(20)

In [24]:
displaymovies(movies, logs['movieId'].values, logs['rating'].values)

In [25]:
displaymovies(movies, recs.index, recs['pred'].values)